In [5]:
from subprocess import check_call, check_output
import os
# import sh

In [13]:
ls = check_output(['ls', '-l', '/'])
print(ls.decode())
print(os.listdir('/'))
mdfind = check_output(['mdfind', '-name', 'test.py'])

total 21
drwxrwxr-x+ 156 root    admin  4992 Dec 18 20:34 Applications
drwxrwxrwt    4 root    wheel   128 Aug 15  2015 DamagedFiles
drwxr-xr-x    2 root    wheel    64 Aug 30  2017 Downloads
drwxrwxr-x    9 root    admin   288 Dec  1  2014 Incompatible Software
drwxr-xr-x+  69 root    wheel  2208 Sep 26 11:26 Library
drwxr-xr-x    2 root    wheel    64 Sep 26 11:18 Network
drwxr-xr-x@   5 root    wheel   160 Sep 21 07:05 System
-rw-r--r--@   1 root    wheel     4 Jun 14  2018 TEST.txt
lrwxr-xr-x    1 root    wheel    49 Oct  3  2014 User Information -> /Library/Documentation/User Information.localized
drwxr-xr-x    6 root    admin   192 Sep 26 11:18 Users
drwxr-xr-x@   6 root    wheel   192 Dec 19 08:50 Volumes
drwxr-xr-x@  37 root    wheel  1184 Dec 13 10:21 bin
drwxrwxr-t    2 root    admin    64 Sep 26 11:18 cores
dr-xr-xr-x    3 root    wheel  4374 Dec 13 10:35 dev
lrwxr-xr-x@   1 root    wheel    11 Sep 26 11:16 etc -> private/etc
dr-xr-xr-x    2 root    wheel     1 Dec 17 19:07 

In [81]:
import ctypes
from ctypes.util import find_library
import numpy
from numpy.testing import assert_allclose
import numba

In [57]:
print(find_library('gsl'))
libm = ctypes.CDLL(find_library('m'))
x = 1
csin = libm.sin
csin.restype = ctypes.c_double
assert_allclose(csin(ctypes.c_double(x)), np.sin(x))
csin.argtypes = (ctypes.c_double,)
assert_allclose(csin(x), np.sin(x))
print(csin(x))

erf = libm.erf
erf.argtypes = (ctypes.c_double,)
erf.restype = ctypes.c_double
erf(x)

/usr/local/lib/libgsl.dylib
0.8414709848078965


0.8427007929497148

In [69]:
libc = ctypes.CDLL(find_library('c'))
from time import time, ctime
ctime(libc.time(None))

'Wed Dec 19 09:44:49 2018'

In [70]:
%load_ext cython

In [90]:
%%cython --annotate

def golden_ratio_cython(int n):
    cdef double s = 1.
    cdef int i
    for i in range(n):
        s = 1. + 1. / s
    return s

In [91]:
def golden_ratio_naive(n):
    s = 1
    for _ in range(n):
        s = 1 + 1 / s
    return s

def golden_ratio_float(n):
    s = 1.
    for _ in range(n):
        s = 1. + 1. / s
    return s

@numba.jit
def golden_ratio_numba(n):
    s = 1.
    for _ in range(n):
        s = 1. + 1. / s
    return s

%timeit golden_ratio_cython(100_000)
%timeit golden_ratio_naive(100_000)
%timeit golden_ratio_float(100_000)
%timeit golden_ratio_numba(100_000)

797 µs ± 51.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
11.9 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.68 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
783 µs ± 8.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [108]:
%%cython

cimport cython
from libc.math cimport erf
cimport numpy as cnp
import numpy as np

# def erf_cy(x):
#     return erf(x)

@cython.boundscheck(False)
@cython.wraparound(False)
def erf_cy(cnp.ndarray[double, ndim=1] a):
    cdef cnp.ndarray[double, ndim=1] b = np.empty_like(a)
    cdef double[:] a_view = a
    cdef double[:] b_view = b
    cdef int i
    cdef int size = a.size
    with nogil:
        for i in range(size):
            b_view[i] = erf(a_view[i])
    return b

In [109]:
from scipy import special

a = np.ones(10**5)
%timeit erf_cy(a)
%timeit special.erf(a)

2.93 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
772 µs ± 67.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
